# Elasticsearch Retrieval with OpenAI

### Overview

In this notebook we will launch elastic search locally and then ask some questions related to 10-K filings of top 10 S&P500 companies

1. Set up enviornment
2. Build search tool to query our ES instance
3. Test the search tool
4. Create OpenAI client with access to the tool
5. Compare OpenAI's response with and without access to the tool

# Imports and start ElasticSearch

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from retriever.util import get_logger
from retriever.searcher.searchtools.elasticsearch import ElasticsearchSearchTool

logger = get_logger()

In [2]:
from retriever.searcher.constants import SEC_FILINGS_SEARCH_TOOL_DESCRIPTION
sec_search_tool = ElasticsearchSearchTool(SEC_FILINGS_SEARCH_TOOL_DESCRIPTION)

Starting Elasticsearch...
Elasticsearch is starting. Please wait a few moments before it becomes available.


In [3]:
tesla_question = "How does Tesla optimize supply chain efficiency?"

In [4]:
tesla_supply_chain = sec_search_tool.search(tesla_question, n_search_results_to_use=3)

2024-02-23 09:34:00,986 - elastic_transport.transport - INFO - POST http://localhost:9200/sec_filings/_search [status:200 duration:0.452s]
2024-02-23 09:34:00,986 - elastic_transport.transport - INFO - POST http://localhost:9200/sec_filings/_search [status:200 duration:0.452s]
2024-02-23 09:34:00,986 - elastic_transport.transport - INFO - POST http://localhost:9200/sec_filings/_search [status:200 duration:0.452s]


In [5]:
print(tesla_supply_chain)


<search_results>
<item index="1">
<page_content>
For
example, a global shortage of semiconductors has been reported since early
2021 and has caused challenges in the manufacturing industry and impacted our
<em>supply</em> <em>chain</em> and production as well.

We are highly dependent on the
services of Elon Musk, Technoking of <em>Tesla</em> and our Chief Executive Officer. We
are highly dependent on the services of Elon Musk, Technoking of <em>Tesla</em> and our
Chief Executive Officer. Although Mr. Musk spends significant time with <em>Tesla</em>
and is highly active in our management, he <em>does</em> not devote his full time and
attention to <em>Tesla</em>. Mr.

There have been and may continue to be
significant <em>supply</em> <em>chain</em> attacks.

We are highly dependent on the
services of Elon Musk, Technoking of <em>Tesla</em> and our Chief Executive Officer. We
are highly dependent on the services of Elon Musk, Technoking of <em>Tesla</em> and our
Chief Executive Officer.

# Now we will analyze different types of responses from the LLM for two different questions

In [6]:
from retriever.client import ClientWithRetrieval
# training data up to Up to Sep 2021
OPENAI_MODEL = "gpt-3.5-turbo"

client = ClientWithRetrieval(api_key=os.environ['OPENAI_API_KEY'], search_tool = sec_search_tool)

/Users/rushilsheth/Documents/portfolio/busco-fin/examples


## Apple 2022 earning

Basic response to the query (no access to the tool).

In [7]:
appl_question = "What was Apple's revenue in 2022?"

In [8]:
basic_response = client.chat.completions.create(
  model=OPENAI_MODEL,
  messages=[
    {"role": "user", "content": appl_question}
  ]
)

2024-02-23 09:34:02,554 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:02,554 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:02,554 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:02,554 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
print('-'*50)
print('Basic response:')
print(appl_question + basic_response.choices[0].message.content)
print('-'*50)

--------------------------------------------------
Basic response:
What was Apple's revenue in 2022?I am unable to provide information about Apple's revenue in 2022 as it is currently the year 2021. To obtain the revenue for 2022, you would need to wait until the end of that year or refer to official financial reports released by Apple.
--------------------------------------------------


Same completion, but give GPT the ability to use the tool when thinking about the response.

In [10]:
augmented_response = client.completion_with_retrieval(
    query=appl_question,
    model=OPENAI_MODEL,
    n_search_results_to_use=3)

2024-02-23 09:34:03,521 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:03,521 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:03,521 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:03,521 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:03,526 - root - INFO - <thinking></thinking>

<search_query>Apple revenue 2022
2024-02-23 09:34:03,526 - root - INFO - <thinking></thinking>

<search_query>Apple revenue 2022
2024-02-23 09:34:03,526 - root - INFO - <thinking></thinking>

<search_query>Apple revenue 2022
2024-02-23 09:34:03,526 - root - INFO - <thinking></thinking>

<search_query>Apple revenue 2022
2024-02-23 09:34:03,528 - root - INFO - Attempting search number 0.
2024-02-23 09:34:03,528 - root - INFO - Attempting search number

In [11]:
print('-'*50)
print('Augmented response:')
print(appl_question + augmented_response)
print('-'*50)

--------------------------------------------------
Augmented response:
What was Apple's revenue in 2022?- Total net sales in 2022 included $7.5 billion of revenue
- $6.7 billion of revenue was recognized in 2021
- $8.2 billion of revenue was recognized in 2023 and included in deferred revenue as of September 24, 2022
- Note 2 in the 2023 Form 10-K discusses revenue recognition at the amount to which Apple expects to be entitled when control of products or services is transferred to customers.
--------------------------------------------------


Basic response to the query (no access to the tool).

## Tesla question

In [12]:
basic_response = client.chat.completions.create(
  model=OPENAI_MODEL,
  messages=[
    {"role": "user", "content": tesla_question}
  ]
)

2024-02-23 09:34:18,983 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:18,983 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:18,983 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:18,983 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [13]:
print('-'*50)
print('Basic response:')
print(tesla_question + basic_response.choices[0].message.content)
print('-'*50)

--------------------------------------------------
Basic response:
How does Tesla optimize supply chain efficiency?1. Vertical Integration: Tesla has strengthened its supply chain by vertically integrating its operations. This means that Tesla owns and operates multiple parts of its supply chain, including manufacturing facilities, distribution centers, and sales channels. By controlling more parts of the supply chain, Tesla can streamline operations and reduce costs.

2. Just-In-Time Manufacturing: Tesla operates on a just-in-time manufacturing model, which means that it only produces products as they are needed. This minimizes excess inventory and reduces the risk of obsolete products, allowing Tesla to operate more efficiently and save on costs.

3. Supplier Relationships: Tesla works closely with its suppliers to establish long-term relationships and partnerships. This helps Tesla build trust with its suppliers and encourages collaboration and communication. By working closely with

In [14]:
augmented_response = client.completion_with_retrieval(
    query=tesla_question,
    model=OPENAI_MODEL,
    n_search_results_to_use=3)

2024-02-23 09:34:19,905 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:19,905 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:19,905 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:19,905 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:34:19,910 - root - INFO - <thinking></thinking>

<search_query>Tesla supply chain efficiency optimization
2024-02-23 09:34:19,910 - root - INFO - <thinking></thinking>

<search_query>Tesla supply chain efficiency optimization
2024-02-23 09:34:19,910 - root - INFO - <thinking></thinking>

<search_query>Tesla supply chain efficiency optimization
2024-02-23 09:34:19,910 - root - INFO - <thinking></thinking>

<search_query>Tesla supply chain efficiency optimization
2024-02-23 09:34:19,918 - root - INF

In [15]:
print('-'*50)
print('Augmented response:')
print(tesla_question + augmented_response)
print('-'*50)

--------------------------------------------------
Augmented response:
How does Tesla optimize supply chain efficiency?- Tesla optimizes supply chain efficiency by integrating the trade-in of customers' existing Tesla or non-Tesla vehicles with the sale of new or used Tesla vehicles.
- The company acquires Tesla and non-Tesla vehicles as trade-ins, which are subsequently remarketed either directly by Tesla or through third parties.
- Despite ongoing supply chain and logistics challenges, Tesla was able to produce and deliver a high number of consumer vehicles in 2022.
- Tesla also faces challenges such as significant supply chain attacks and global shortages of key components like semiconductors, which impact their production and supply chain operations.
--------------------------------------------------
